## NLP 

研究的领域非常广泛，凡是跟语言学有关的内容都属于 NLP 的范畴。

一般来说，较为多见的语言学的方向包括：词干提取、词形还原、分词、词性标注、命名实体识别、语义消歧、句法分析、指代消解、篇章分析等方面。


![image13.png](mdfiles/image13.png)


搜索引擎要对你输入的内容（query）进行解析，这就涉及到了之前提到的分词、命名实体识别、语义消歧等内容，当然还涉及到了 query 纠错，因为你错误地输入了拼音而非汉字，需要改写成正确的形式。

### 分词

中文单词和单词之间除了标点符号没有别的隔断。这就给程序理解文本带来了一定的难度，分词的需求也应运而生。

深度学习已经对分词的依赖越来越小，可以通过 Word Embedding 等方式对字符（token）级的文字进行表示，但是分词的地位不会降低，单词、词组级别的文本表示仍旧有非常多的应用场景。

开源或者免费的 NLP 分词工具，比如 jieba、HanLP、THULAC

#### jieba 分词


In [ ]:
%pip install -q jieba


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 103.4 MB/s  0:00:000:00:01
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'jieba' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'jieba'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314509 sha256=3d9285c1c0bee9cd369d18d9f630e6e1715792dee72675abe68ff489e2a3ca6a
  Stored in directory: /home/uceeqz4/.cache/pip/wheels/ac/60/cf/538a1f183409caf1fc136b5d2c2dee329001ef6da2c5084bef
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [2]:
import jieba
text = "极客时间棒呆啦"
# jieba.cut得到的是generator形式的结果
seg = jieba.cut(text)  
print(' '.join(seg)) 

# Get： 极客 时间 棒呆 啦

/home/uceeqz4/.conda/envs/pytorch311/lib/python3.11/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


极客 时间 棒呆 啦


In [ ]:
# jieba 还提供了词性标注的结果（pos）
import jieba.posseg as posseg
text = "今年天气好晴朗，处处好风光"
# 形如pair('word, 'pos')的结果
seg = posseg.cut(text)  
print([se for se in seg]) 

# m: 数词
# d: 副词
# v: 动词
# n: 普通名词
# r: 代词
# a: 形容词
# x: 非语素字


[pair('今年', 't'), pair('天气', 'n'), pair('好', 'a'), pair('晴朗', 'a'), pair('，', 'x'), pair('处处', 'v'), pair('好', 'a'), pair('风光', 'n')]


#### One-hot

假定所有的文字一共有 N 个单词（也可以是字符），我们可以将每个单词赋予一个单独的序号 id，那么对于任意一个单词，我们都可以采用一个 N 位的列表（向量）对其进行表示。在这个表示中，只需要将这个单词对应序号 id 的位置为 1，其他位置为 0 即可。

比方说，我们词典大小为 10000，“极客”这个单词的序号 id 为 666，那么我们就需要建立一个 10000 长度的向量


在 UTF-8 编码中，中文字符有两万多个，词语数量更是天文数字，那么我们仅用字符的方式，每个字就需要两万多维度的数据来表示。推算一下，如果有一篇一万字的文章，这个数据量就很可怕了。

为了进一步的压缩数据的体积，可以只使用一个向量表示文章中所有的单词，例如前面的例子，我们仍旧建立一个 10000 维的向量，把文章中出现过的所有单词的对应位置置为 1，其余为 0。


count-based 进一步压缩体积
对每一个出现的单词的序号 id 以及出现过的次数进行统计，这样一来，“极客时间”我们只需要两个 k-v 对的 dict 即可表示: {3:1, 665:1}。

这种表示方法在 SVM、树模型等多个算法包中被广泛采用，但是新的问题就是不能表述单词的语序信息

还有其他很多word Embedding 的方法


### 关键词的提取

基于统计特征的方法

TF-IDF（term frequency–inverse document frequency，词频 - 逆向文件频率）

一个单词在文件中出现的次数越多，它的重要性越高；但它在语料库中出现的频率越高，它的重要性反而越小。

TF-IDF 中，词频（TF）表示关键字在文本中出现的频率。而逆向文件频率 (IDF)  是由包含该词语的文件的数目除以总文件数目得到的，一般情况下还会取对数对结果进行缩放。

$$ 
TF = \frac{\text{某词在该文件中出现的次数}}{\text{该文件的总词数}}
$$

$$
IDF = \log{\frac{\text{总文件数}}{\text{包含该词的文件数 + 1}}}
$$

这里为什么分母要加 1 呢？这是为了避免分母为 0 的情况。得到了 TF 和 IDF 之后，我们将两者相乘，就得到了 TF-IDF 了。

对单词出现的次数以及分布进行数学上的统计，从而发现相应的规律和重要性（权重），并以此作为关键词提取的依据。

NLTK

In [18]:
import nltk
from nltk import word_tokenize, TextCollection
nltk.download('punkt')
nltk.download('punkt_tab')

sents = ['i like jike', 'i want to eat apple', 'i like lady gaga']
# 分词
sents = [word_tokenize(sent) for sent in sents]

# 构建语料库
corpus = TextCollection(sents)

# 选一个文档，比如第一个句子
doc = sents[0]

# 计算TF、IDF、TF-IDF
tf = corpus.tf('like', doc)
idf = corpus.idf('like')
tf_idf = corpus.tf_idf('like', doc)

print(f'TF: {tf}, IDF: {idf}, TF-IDF: {tf_idf}')


TF: 0.3333333333333333, IDF: 0.4054651081081644, TF-IDF: 0.13515503603605478


[nltk_data] Downloading package punkt to /home/uceeqz4/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/uceeqz4/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


| 名称         | 全称                         | 中文含义  | 作用                   |
| ---------- | -------------------------- | ----- | -------------------- |
| **TF**     | Term Frequency             | 词频    | 表示某个词在一篇文档中出现的“频繁程度” |
| **IDF**    | Inverse Document Frequency | 逆文档频率 | 衡量某个词在整个语料库中是否“普遍”   |
| **TF-IDF** | TF × IDF                   | 综合权重  | 表示该词在该文档中对“区分主题”的重要性 |


### 基于词图模型的关键词提取

基于词图模型的关键词提取。

我们首先要构建文本一个图结构，用来表示语言的词语网络。然后对语言进行网络图分析，在这个图上寻找具有重要作用的词或者短语，即关键词。

TextRank 来自 PageRank 算法

PageRank 是 Google 用来对网页进行排序的算法。它的核心思想是：一个网页的重要性由指向它的其他网页的重要性决定。
1. 如果一个网页被很多其他网页链接到的话，就说明这个网页比较重要，也就是 PageRank 值会相对较高。
2. 如果一个 PageRank 值很高的网页，链接到一个其他的网页，那么被链接到的网页的 PageRank 值会相应地因此而提高。

对于TextRank:

![image14.png](mdfiles/image14.png)



In [ ]:
 
# jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'), withFlag=False)

In [2]:
# TextRank 示例：从一段中文中抽取关键词
from jieba import analyse

sentence = (
    "极客时间是一个专注于程序员成长的知识服务平台。本文介绍自然语言处理中的关键词提取方法，"
    "包括基于统计的TF-IDF和基于图模型的TextRank。我们将使用jieba的TextRank从一段中文里抽取关键词，"
    "并对结果进行简单分析。关键词应该能反映主题，比如‘关键词提取’、‘文本’、‘算法’、‘图模型’等。"
 )

# 定义一个较全面的词性集合来模拟“不过滤词性”
ALL_POS = (
    "n","nr","ns","nt","nz","nl","ng",  # 名词及细分
    "v","vd","vn","vq","vg",            # 动词及细分
    "a","ad","an","ag",                    # 形容词及细分
    "x"                                      # 其他（符号/未知）
 )

# 1) 默认（只保留 ns/n/vn/v 这些词性），返回关键词列表
keys_default = analyse.textrank(
    sentence,
    topK=20,
    withWeight=False,
    allowPOS=("ns", "n", "vn", "v"),
    withFlag=False,
 )
print("默认词性过滤的关键词:")
print(keys_default)

# 2) 返回带权重的关键词 (前10个)
keys_weighted = analyse.textrank(
    sentence,
    topK=10,
    withWeight=True,
    allowPOS=("ns", "n", "vn", "v"),
    withFlag=False,
 )
print("\n带权重的关键词:")
for word, score in keys_weighted:
    print(f"{word}\t{score:.4f}")

# 3) 使用更全面的词性集合（近似“不过滤”），看看有什么变化
keys_allpos = analyse.textrank(
    sentence,
    topK=10,
    withWeight=False,
    allowPOS=ALL_POS,   # 用更宽的集合代替 None，避免 TypeError
    withFlag=False,
 )
print("\n更宽词性集合下的关键词:")
print(keys_allpos)


默认词性过滤的关键词:
['关键词', '提取', '专注', '程序员', '成长', '主题', '知识', '服务平台', '方法', '包括', '分析', '比如', '应该', '反映', '介绍', '处理', '文本', '进行', '结果', '极客']

带权重的关键词:
关键词	1.0000
提取	0.5471
专注	0.3940
程序员	0.3571
成长	0.3541
主题	0.3520
知识	0.3491
服务平台	0.3481
方法	0.3205
包括	0.3187

更宽词性集合下的关键词:
['关键词', '提取', '专注', '程序员', '成长', '知识', '服务平台', '简单', '应该', '分析']


### 基于主题模型的关键词提取    

LDA（Latent Dirichlet Allocation）文档主题生成模型，是最典型的基于主题模型的算法。通过对文档进行主题建模，可以提取出每个主题的关键词，从而实现对文档的理解和分析。

In [4]:
# pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 137.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 146.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [gensim]2m3/4 [gensim]pen]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gensim import corpora, models
import jieba.posseg as jp
import jieba

texts = [line.strip() for line in open('input.txt', 'r')]
# 老规矩，先分词
words_list = []
for text in texts:
  words = [w.word for w in jp.cut(text)]
  words_list.append(words)

# 构建文本统计信息, 遍历所有的文本，为每个不重复的单词分配序列id，同时收集该单词出现的次数
dictionary = corpora.Dictionary(words_list)

# 构建语料，将dictionary转化为一个词袋。
# corpus是一个向量的列表，向量的个数就是文档数。你可以输出看一下它内部的结构是怎样的。
corpus = [dictionary.doc2bow(words) for words in words_list]

# 开始训练LDA模型
lda_model = models.ldamodel.LdaModel(corpus=corpus, num_topics=8, id2word=dictionary, passes=10)

在训练环节中，num_topics 代表生成的主题的个数。id2word 即为 dictionary，它把 id 都映射成为字符串。passes 相当于深度学习中的 epoch，表示模型遍历语料库的次数